In [1]:
from sqlalchemy import create_engine
import pandas as pd 
import json
pd.set_option('display.max_columns', 500)
with open('/home/douglas/postgres_credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']
    address = data['address']

conn = create_engine('postgresql://{}:{}@{}:5432/ex_normalize'.format(username, password, address))

/home/douglas/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Explore with CSV

Look at structure of data set using pandas.
Skip ahead to next section 'Normalize Database' for normalization steps with SQL. 

In [2]:
orders = pd.read_csv('/var/tmp/SQL_practice/07_normalize_superstore/orders.csv')

In [3]:
orders.head()

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Name,Product Base Margin,Country,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Pens & Art Supplies,Wrap Bag,SANFORD Liquid Accent™ Tank-Style Highlighters,0.54,United States,West,Washington,Anacortes,98221,2015-01-07,2015-01-08,4.5600,4,13.01,88522
1,20228,Not Specified,0.02,500.98,26.00,5,Ronnie Proctor,Delivery Truck,Home Office,Furniture,Chairs & Chairmats,Jumbo Drum,Global Troy™ Executive Leather Low-Back Tilter,0.60,United States,West,California,San Gabriel,91776,2015-06-13,2015-06-15,4390.3665,12,6362.85,90193
2,21776,Critical,0.06,9.48,7.29,11,Marcus Dunlap,Regular Air,Home Office,Furniture,Office Furnishings,Small Pack,"DAX Two-Tone Rosewood/Black Document Frame, De...",0.45,United States,East,New Jersey,Roselle,7203,2015-02-15,2015-02-17,-53.8096,22,211.15,90192
3,24844,Medium,0.09,78.69,19.99,14,Gwendolyn F Tyson,Regular Air,Small Business,Furniture,Office Furnishings,Small Box,Howard Miller 12-3/4 Diameter Accuwave DS ™ Wa...,0.43,United States,Central,Minnesota,Prior Lake,55372,2015-05-12,2015-05-14,803.4705,16,1164.45,86838
4,24846,Medium,0.08,3.28,2.31,14,Gwendolyn F Tyson,Regular Air,Small Business,Office Supplies,Pens & Art Supplies,Wrap Bag,Newell 321,0.56,United States,Central,Minnesota,Prior Lake,55372,2015-05-12,2015-05-13,-24.0300,7,22.23,86838


In [4]:
orders.columns

Index(['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost',
       'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment',
       'Product Category', 'Product Sub-Category', 'Product Container',
       'Product Name', 'Product Base Margin', 'Country', 'Region',
       'State or Province', 'City', 'Postal Code', 'Order Date', 'Ship Date',
       'Profit', 'Quantity ordered new', 'Sales', 'Order ID'],
      dtype='object')

In [5]:
orders.shape

(1952, 25)

In [6]:
orders.dtypes

Row ID                    int64
Order Priority           object
Discount                float64
Unit Price              float64
Shipping Cost           float64
Customer ID               int64
Customer Name            object
Ship Mode                object
Customer Segment         object
Product Category         object
Product Sub-Category     object
Product Container        object
Product Name             object
Product Base Margin     float64
Country                  object
Region                   object
State or Province        object
City                     object
Postal Code               int64
Order Date               object
Ship Date                object
Profit                  float64
Quantity ordered new      int64
Sales                   float64
Order ID                  int64
dtype: object

In [7]:
for c in orders.columns:
    print(c, orders[c].nunique())

Row ID 1951
Order Priority 6
Discount 13
Unit Price 597
Shipping Cost 497
Customer ID 1130
Customer Name 1130
Ship Mode 3
Customer Segment 4
Product Category 3
Product Sub-Category 17
Product Container 7
Product Name 913
Product Base Margin 51
Country 1
Region 4
State or Province 49
City 869
Postal Code 981
Order Date 179
Ship Date 187
Profit 1897
Quantity ordered new 76
Sales 1922
Order ID 1365


In [8]:
# orders.groupby('Customer ID')['Postal Code'].nunique()
# all 1's
# customers have unique addresses but what if have multiple addresses?
# create own table for addresses

In [9]:
#orders.groupby('Customer ID')['Order ID'].nunique()

In [10]:
# look at one person with multiple orders
orders[orders['Customer ID'] == 32]
# note same address

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Name,Product Base Margin,Country,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
19,23509,High,0.08,34.99,7.73,32,Matthew Berman,Regular Air,Corporate,Office Supplies,Pens & Art Supplies,Small Box,Hunt Boston® Vacuum Mount KS Pencil Sharpener,0.59,United States,West,Oregon,Grants Pass,97526,2015-02-22,2015-02-23,144.6900,13,424.68,89199
20,23612,High,0.01,17.98,8.51,32,Matthew Berman,Regular Air,Corporate,Technology,Office Machines,Medium Box,Canon P1-DHIII Palm Printing Calculator,0.40,United States,West,Oregon,Grants Pass,97526,2015-03-27,2015-03-28,-35.8788,2,40.17,89200
21,23278,Medium,0.09,125.99,7.69,32,Matthew Berman,Express Air,Corporate,Technology,Telephones and Communication,Small Box,StarTAC 3000,0.59,United States,West,Oregon,Grants Pass,97526,2015-01-20,2015-01-22,209.9970,8,783.55,89202
22,19355,Low,0.06,205.99,8.99,32,Matthew Berman,Regular Air,Corporate,Technology,Telephones and Communication,Small Box,TimeportP7382,0.56,United States,West,Oregon,Grants Pass,97526,2015-03-12,2015-03-19,3568.0960,22,3838.14,89203


In [11]:
# Sales doesn't add up with multiplying quantity by unit price?
205.99 * 22 * 0.94

4259.8732

In [12]:
money = orders[['Discount', 'Unit Price', 'Shipping Cost', 'Product Base Margin', 'Profit', 'Quantity ordered new', 'Sales']]

In [13]:
# make a calc column to check if other rows add up
money['disc_total'] = money['Unit Price'] * money['Quantity ordered new'] * (1 - money['Discount']) 
money.head()

/home/douglas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Discount,Unit Price,Shipping Cost,Product Base Margin,Profit,Quantity ordered new,Sales,disc_total
0,0.01,2.84,0.93,0.54,4.5600,4,13.01,11.2464
1,0.02,500.98,26.00,0.60,4390.3665,12,6362.85,5891.5248
2,0.06,9.48,7.29,0.45,-53.8096,22,211.15,196.0464
3,0.09,78.69,19.99,0.43,803.4705,16,1164.45,1145.7264
4,0.08,3.28,2.31,0.56,-24.0300,7,22.23,21.1232


In [14]:
# Nope, will need more info about the data collected.

## Pick out which columns belong to each table

In [15]:
# Table 1: Orders
table1 = ['Row ID',
'Order Priority',
'Discount', 
'Shipping Cost',
'Customer ID', 
'Customer Segment', # not unique to customer_id- depends on order          
'Ship Mode', 
'PRODUCT_ID', #(window over Product name)
'ADDRESS_ID', # (window over address)
'Order Date', 
'Ship Date', 
'Profit', 
'Quantity ordered new', 
'Sales', 
'Order ID' ]

In [16]:
# Table 2: Customer 
table2 = ['Customer ID', 
'Customer Name']

In [17]:
# Table 3: Address 
table3 = ['ADDRESS_ID'
'Country', 
'Region',
'State or Province', 
'City', 
'Postal Code',
'Customer ID']

In [18]:
# Table 4: Product
table4 = ['PRODUCT ID', 
'Unit Price', 
'Product Category', 
'Product Sub-Category', 
'Product Container',
'Product Name', 
'Product Base Margin']

# Normalize database

In [19]:
# even full address isn't unique, so need customer_id in DISTINCT 
orders.groupby(['Country', 'Region', 'State or Province', 'City', 'Postal Code'])['Customer ID'].nunique().value_counts()

1    859
2    114
3      7
4      4
6      1
Name: Customer ID, dtype: int64

In [20]:
orders.groupby(['City', 'Postal Code'])['Customer ID'].nunique().value_counts()
# only city and postal code is not enough to distinguish

1    859
2    114
3      7
4      4
6      1
Name: Customer ID, dtype: int64

In [21]:
# use all cols to create ADDRESS_ID 
pd.read_sql_query('''
SELECT *, row_number() OVER (ORDER BY customer_id) AS ADDRESS_ID
FROM  (SELECT DISTINCT customer_id, country, region, state, city, postal_code 
FROM orders
ORDER BY customer_id) s
LIMIT 5
''', conn)

,customer_id,country,region,state,city,postal_code,address_id
0,3,United States,West,Washington,Anacortes,98221,1
1,5,United States,West,California,San Gabriel,91776,2
2,11,United States,East,New Jersey,Roselle,7203,3
3,14,United States,Central,Minnesota,Prior Lake,55372,4
4,15,United States,East,New York,Smithtown,11787,5


In [22]:
# double check products are not under two categories for some reason
orders.groupby('Product Name')['Product Category'].nunique().value_counts()

1    913
Name: Product Category, dtype: int64

1. Product_base_margin can be diff for the same name product  
Ex. 'Keytronic 105-Key Spanish Keyboard'

2. 'Metal Folding Chairs, Beige, 4/Carton' possible error  
Unit price have a magnitude difference despite same text description
3. Figured out after initially trying only product_category and product_name.  
927 vs 913 rows with all columns used, so only 14 items with issues

4. Going to select distinct all relevant columns for now. With a real dataset, I would try to ask for more information and try to clean up the errors since in theory, these columns ought to be be unique to the product.

In [23]:
# see if query for PRODUCT ID works
prods = pd.read_sql_query('''
SELECT *, row_number() OVER (ORDER BY product_name, unit_price, product_base_margin) AS PRODUCT_ID
FROM  (SELECT DISTINCT unit_price,
                        product_category, 
                        product_subcategory,
                        product_container,
                        product_name, 
                        product_base_margin
FROM orders
ORDER BY product_category, product_name) s

''', conn)

In [24]:
len(prods)

927

In [25]:
prods.groupby(['unit_price',
                        'product_category', 
                        'product_subcategory',
                       'product_container',
                        'product_name', 
                        'product_base_margin'])['product_id'].nunique()
# 921

unit_price  product_category  product_subcategory             product_container  product_name                                                                 product_base_margin
1.14        Office Supplies   Rubber Bands                    Wrap Bag           OIC Thumb-Tacks                                                              0.38                   1
1.26        Office Supplies   Rubber Bands                    Wrap Bag           Bagged Rubber Bands                                                          0.81                   1
1.48        Office Supplies   Rubber Bands                    Wrap Bag           Binder Clips by OIC                                                          0.37                   1
1.60        Office Supplies   Pens & Art Supplies             Wrap Bag           Sanford Pocket Accent® Highlighters                                          0.42                   1
1.68        Office Supplies   Pens & Art Supplies             Wrap Bag           Newell 30

In [26]:
prods.head()

,unit_price,product_category,product_subcategory,product_container,product_name,product_base_margin,product_id
0,15.74,Office Supplies,Envelopes,Small Box,"#10-4 1/8"" x 9 1/2"" Premium Diagonal Seam Enve...",0.40,1
1,8.74,Office Supplies,Envelopes,Small Box,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",0.38,2
2,7.64,Office Supplies,Envelopes,Small Box,"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes",0.36,3
3,15.67,Office Supplies,Envelopes,Small Box,"#10 White Business Envelopes,4 1/8 x 9 1/2",0.38,4
4,19.98,Furniture,Office Furnishings,Small Box,12-1/2 Diameter Round Wall Clock,0.49,5


In [27]:
# LEFT JOIN TO see where nulls are popping up
nulls = pd.read_sql_query('''SELECT o.*,  p.product_id
FROM orders o
LEFT JOIN (SELECT *, row_number() OVER (ORDER BY product_name) AS PRODUCT_ID
            FROM  (SELECT DISTINCT unit_price,
                    product_category, 
                    product_subcategory,
                    product_container,
                    product_name, 
                    product_base_margin
            FROM orders
            ORDER BY product_category, product_name) s
        ) p
ON o.product_category = p.product_category
AND o.product_subcategory = p.product_subcategory
AND o.product_container = p.product_container
AND o.product_name = p.product_name
AND o.product_base_margin = p.product_base_margin
AND o.unit_price = p.unit_price ''', conn)

In [28]:
orders[orders['Customer ID'] == 1281] 
# turns out there are NaNs in product_base_margin

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Name,Product Base Margin,Country,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
734,22125,Low,0.10,238.40,24.49,1281,Pauline Denton,Regular Air,Small Business,Furniture,Chairs & Chairmats,Large Box,Safco Contoured Stacking Chairs,NaN,United States,Central,Indiana,Vincennes,47591,2015-01-24,2015-01-26,875.2844,8,1774.50,89112
735,22126,Low,0.03,199.99,24.49,1281,Pauline Denton,Express Air,Small Business,Technology,Copiers and Fax,Large Box,Canon PC-428 Personal Copier,0.46,United States,Central,Indiana,Vincennes,47591,2015-01-24,2015-01-26,727.7361,5,1054.69,89112


In [29]:
len(orders[orders['Product Base Margin'].isnull()]) # all are in product_base_margin

16

In [30]:
#nulls[nulls['product_base_margin'].isnull()]

### IMPORTANT: FILL NULLS with 0 in original table orders

In [31]:
# conn.execute('''
# UPDATE orders
# SET product_base_margin = 0
# WHERE product_base_margin IS NULL''')

In [32]:
# join these new id cols to original table
pd.read_sql_query('''
SELECT count(*) --o.*, a.address_id, p.product_id
FROM orders o
JOIN (SELECT *, row_number() OVER (ORDER BY customer_id) AS ADDRESS_ID
        FROM  (SELECT DISTINCT customer_id, country, region, state, city, postal_code 
                FROM orders
                ORDER BY customer_id) s
        ) a
ON o.customer_id = a.customer_id
    AND o.city = a.city
    AND o.postal_code = a.postal_code 
-- for completeness, should do JOIN ON all, but this gets unique enough for this data even if a person has two addresses
JOIN (SELECT *, row_number() OVER (ORDER BY product_name) AS PRODUCT_ID
            FROM  (SELECT DISTINCT unit_price,
                    product_category, 
                    product_subcategory,
                    product_container,
                    product_name, 
                    product_base_margin
            FROM orders
            ORDER BY product_category, product_name) s
        ) p
ON o.product_category = p.product_category
AND o.product_name = p.product_name
AND o.product_base_margin = p.product_base_margin
AND o.unit_price = p.unit_price
''', conn)
# note get all 1952 rows now

,count
0,1952


In [33]:
# CREATE VIEW of full table with new cols
conn.execute('''CREATE OR REPLACE VIEW full_orders AS 
(SELECT o.*, a.address_id, p.product_id
FROM orders o
JOIN (SELECT *, row_number() OVER (ORDER BY customer_id) AS ADDRESS_ID
        FROM  (SELECT DISTINCT customer_id, country, region, state, city, postal_code 
                FROM orders
                ORDER BY customer_id) s
        ) a
ON o.customer_id = a.customer_id
    AND o.city = a.city
    AND o.postal_code = a.postal_code 
-- for completeness, should do JOIN ON all, but this gets unique enough for this data even if a person has two addresses
JOIN (SELECT *, row_number() OVER (ORDER BY product_name) AS PRODUCT_ID
            FROM  (SELECT DISTINCT unit_price,
                    product_category, 
                    product_subcategory,
                    product_container,
                    product_name, 
                    product_base_margin
            FROM orders
            ORDER BY product_category, product_name) s
        ) p
ON o.product_category = p.product_category
AND o.product_name = p.product_name
AND o.product_base_margin = p.product_base_margin
AND o.unit_price = p.unit_price)''')

In [34]:
pd.read_sql_query("SELECT COUNT(*) FROM full_orders", conn)

,count
0,1952


With view full_orders created, now create the 4 tables and insert the appropriate columns to populate each table.

### Table 1: norm_orders

In [35]:
conn.execute('''
CREATE TABLE IF NOT EXISTS norm_orders(
row_id INTEGER,
order_priority TEXT,
discount REAL,
shipping_cost REAL,
customer_id INTEGER,
customer_segment TEXT,
ship_mode TEXT,
product_id INTEGER,
address_id INTEGER,
order_date TIMESTAMP,
ship_date TIMESTAMP,
profit REAL,
quantity_ordered_new INTEGER,
sales REAL,
order_id INTEGER);
''')

In [36]:
# # insert appropriate columns
# conn.execute('''
# INSERT INTO norm_orders
#     SELECT row_id,
#         order_priority,
#         discount,
#         shipping_cost,
#         customer_id,
#         customer_segment,
#         ship_mode,
#         product_id ,
#         address_id ,
#         order_date,
#         ship_date,
#         profit,
#         quantity_ordered_new,
#         sales,
#         order_id
#     FROM full_orders''')

### Table 2: customers

In [37]:
conn.execute('''
CREATE TABLE IF NOT EXISTS customers(
customer_id INTEGER NOT NULL,
customer_name TEXT,
PRIMARY KEY (customer_id));
''')

In [38]:
# conn.execute('''
# INSERT INTO customers
#     SELECT DISTINCT customer_id,
#         customer_name
#     FROM full_orders''')

In [39]:
orders[orders['Customer ID'] == 1527]
# customer_id can have more than one customer_segment, so it does belong in the order table after all
# edited cells above

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Name,Product Base Margin,Country,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
856,24974,Critical,0.03,30.98,8.99,1527,Neil Parker,Express Air,Small Business,Office Supplies,Pens & Art Supplies,Small Pack,"Boston School Pro Electric Pencil Sharpener, 1670",0.58,United States,South,Alabama,Decatur,35601,2015-01-09,2015-01-11,0.510,5,162.39,86813
857,22253,Low,0.03,65.99,5.26,1527,Neil Parker,Regular Air,Home Office,Technology,Telephones and Communication,Small Box,8860,0.56,United States,South,Alabama,Decatur,35601,2015-03-30,2015-04-09,-52.248,23,1316.03,86814
858,21455,Low,0.09,50.98,6.50,1527,Neil Parker,Regular Air,Home Office,Technology,Computer Peripherals,Small Box,Microsoft Natural Multimedia Keyboard,0.73,United States,South,Alabama,Decatur,35601,2015-05-21,2015-05-28,70.176,28,1395.41,86815


### Table 3: addresses

In [40]:
conn.execute('''
CREATE TABLE IF NOT EXISTS addresses(
    address_id INTEGER NOT NULL,
    country TEXT,
    region TEXT,
    state TEXT,
    city TEXT,
    postal_code TEXT,
    customer_id INTEGER NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    PRIMARY KEY (address_id)
    );
''')

In [41]:
# conn.execute('''
# INSERT INTO addresses
#     SELECT DISTINCT 
#         address_id,
#         country, 
#         region,
#         state,
#         city,
#         postal_code,
#         customer_id
#     FROM full_orders''')

### Table 4: products

In [42]:
conn.execute('''
CREATE TABLE IF NOT EXISTS products(
    product_id INTEGER NOT NULL,
    unit_price REAL,
    product_category TEXT,
    product_subcategory TEXT,
    product_container TEXT,
    product_name TEXT,
    product_base_margin REAL,
    PRIMARY KEY (product_id)
    );
''')

In [43]:
# conn.execute('''
# INSERT INTO products
#     SELECT DISTINCT 
#         product_id,
#         unit_price,
#         product_category,
#         product_subcategory,
#         product_container,
#         product_name,
#         product_base_margin
#     FROM full_orders''')

In [44]:
# query used to look at errors 
# pd.read_sql_query('''
# SELECT   product_id,
#         unit_price,
#         product_category,
#         product_subcategory,
#         product_container,
#         product_name,
#         product_base_margin
#         FROM full_orders
#         WHERE product_id = 567 ''', conn)

In [45]:
reconstructed = pd.read_sql_query('''
SELECT * FROM 
norm_orders n
JOIN customers c
ON n.customer_id = c.customer_id
JOIN addresses a
ON n.address_id = a.address_id
JOIN products p
ON n.product_id = p.product_id''', conn)

In [46]:
len(reconstructed)

1952

To be more precise, need to deselect the duplicate ID columns and then compare with original table to see if the table was properly reconstructed but the same length is a good sign.